In [0]:
bronze_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/FileStore/sample_data/sales.csv")
)

In [0]:
bronze_df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("bronze_sales")

---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-5846833739128886>, line 4
      1 bronze_df.write \
      2     .format("delta") \
      3     .mode("append") \
----> 4     .saveAsTable("bronze_sales")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1554     req.user_context.user_id = self._user_id
   

In [0]:
from pyspark.sql.functions import col

silver_df = (
    spark.table("bronze_sales")
    .filter(col("Order ID").isNotNull())
    .dropDuplicates(["Order ID"])
    .withColumn("Sales", col("Sales").cast("double"))
    .withColumn("Profit", col("Profit").cast("double"))
)

In [0]:
silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_sales")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5846833739128888>, line 4
      1 silver_df.write \
      2     .format("delta") \
      3     .mode("overwrite") \
----> 4     .saveAsTable("silver_sales")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1554     req.user_context.user_id = self._user_id


In [0]:
from pyspark.sql.functions import sum

gold_df = (
    spark.table("silver_sales")
    .groupBy("Category")
    .agg(
        sum("Sales").alias("total_sales"),
        sum("Profit").alias("total_profit")
    )
)

In [0]:
gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_sales_category")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5846833739128890>, line 4
      1 gold_df.write \
      2     .format("delta") \
      3     .mode("overwrite") \
----> 4     .saveAsTable("gold_sales_category")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1554     req.user_context.user_id = self._use

In [0]:
MERGE INTO silver_sales t
USING bronze_sales s
ON t.`Order ID` = s.`Order ID`
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

  File <command-5846833739128891>, line 1
    MERGE INTO silver_sales t
          ^
SyntaxError: invalid syntax
